# kidney

In [1]:
setwd('/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/')
library(Matrix)
library(readr)
source('/gpfs/gibbs/pi/zhao/xs272/data_preprocess/utils.r')
source('/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/scripts/utils.r')

Attaching SeuratObject

Attaching sp

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, t

## 1. Unimodal RNA preprocessing

In [2]:
kidney.rna.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseKidney/RNA'
kidney.rna.counts <- readRDS(file.path(kidney.rna.path, 'GSE157079_P0_adult_counts.rds'))

In [3]:
kidney.rna.meta <- read.csv(file.path(kidney.rna.path, 'GSE157079_P0_adult_clusters_filtered.csv'))

In [4]:
kidney.rna.annot <- kidney.rna.meta$annotation
names(kidney.rna.annot) <- kidney.rna.meta$barcodes

In [5]:
print.names.match(kidney.rna.meta$barcodes, colnames(kidney.rna.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "43410 43636 43410 FALSE"


In [6]:
kidney.rna.counts <- kidney.rna.counts[, kidney.rna.meta$barcodes]
kidney.rna.annot <- kidney.rna.annot[kidney.rna.meta$barcodes]

In [7]:
kidney.rna.counts ## ! did not fix barcode or gene names yet

   [[ suppressing 32 column names 'Ksp_AAACCTGAGAAGGGTA', 'Ksp_AAACCTGAGACTGGGT', 'Ksp_AAACCTGAGAGTCGGT' ... ]]

   [[ suppressing 32 column names 'Ksp_AAACCTGAGAAGGGTA', 'Ksp_AAACCTGAGACTGGGT', 'Ksp_AAACCTGAGAGTCGGT' ... ]]

   [[ suppressing 32 column names 'Ksp_AAACCTGAGAAGGGTA', 'Ksp_AAACCTGAGACTGGGT', 'Ksp_AAACCTGAGAGTCGGT' ... ]]



31053 x 43410 sparse Matrix of class "dgCMatrix"
                                                                                             
Xkr4           .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Gm1992         .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Gm37381        .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Rp1            .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Sox17          .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Gm37323        .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Mrpl15         .  .  1 .  .  .  .  1  . .  1  . . .  .  . .  .  1  . .  .  . . . .  .  .  . .
Lypla1         1  .  . .  2  2  1  .  1 .  1  . . 1  2  . .  4  .  1 .  .  . 1 . .  1  .  . .
Gm37988        .  .  . .  .  .  .  .  . .  .  . . .  .  . .  .  .  . .  .  . . . .  .  .  . .
Tcea1      

In [ ]:
kidney.rna.path <- 'kidney/RNA'
write.counts.transpose(kidney.rna.counts, kidney.rna.path)
write.annotation(kidney.rna.annot, kidney.rna.path)

## 2. Unimodal ATAC counts

In [22]:
kidney.atac.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseKidney/ATAC'
kidney.atac.counts <- readMM(file.path(kidney.atac.path, 'GSE157079_snATAC_cell_by_peak_matrix.txt.gz'))
kidney.atac.counts <- t(kidney.atac.counts)

In [23]:
kidney.atac.peak.table <- read.csv(file.path(kidney.atac.path, 'GSE157079_snATAC_peak_list.csv.gz'))
kidney.atac.meta <- read.csv(file.path(kidney.atac.path, 'GSE157079_snATAC_metadata_filtered.csv'))

In [24]:
kidney.atac.counts <- kidney.atac.counts[, kidney.atac.meta$X] # the meta file has filtered out some cells!
kidney.atac.barcodes <- paste(kidney.atac.meta$samples, kidney.atac.meta$barcodes, sep='-')
kidney.atac.counts <- set.names(counts = kidney.atac.counts,  rowname = format.peaks(kidney.atac.peak.table$name, to.replace = ':', change.underscore = TRUE), colname = kidney.atac.meta$barcodes, allow.duplicate = TRUE) # don't add the samples yet, to align with the fragment files.

In [25]:
kidney.atac.annot <- kidney.atac.meta$annotation
names(kidney.atac.annot) <- kidney.atac.barcodes

In [ ]:
kidney.atac.path <- 'kidney/ATAC'
kidney.atac.counts.uniq <- set.names(counts = kidney.atac.counts,  rowname = format.peaks(kidney.atac.peak.table$name, to.replace = ':', change.underscore = TRUE), colname = kidney.atac.barcodes, allow.duplicate = FALSE)
write.counts.transpose(kidney.atac.counts.uniq, kidney.atac.path)
write.annotation(kidney.atac.annot, kidney.atac.path)

## 3. Unimodal ATAC fragment file

In [30]:
in.frag.path <- "/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseKidney/ATAC/fragments/"
out.frag.path <- 'tmp/kidney/'

In [31]:
system(paste('bash frag_prep_kidney.sh -i', in.frag.path, '-o', out.frag.path, sep=' '))

In [32]:
frag.files <- c(P0_batch1 = 'P0_1', 
                P0_batch2 = 'P0_2', 
                P21_batch1 = '3_week', 
                P56_batch1 = '8_week_1', 
                P56_batch2 = '8_week_2')

## 4. Unimodal gene activity (scATAC-seq)

In [35]:
library(Signac)
library(EnsDb.Mmusculus.v79)
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotations) <- 'UCSC'

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

In [36]:
kidney.atac.ga.list <- list()
for (tissue in names(frag.files)) {
    selected.id <- which(kidney.atac.meta$samples == tissue)
    selected.kidney.count <- kidney.atac.counts[, selected.id]
    stopifnot(!any(duplicated(colnames(selected.kidney.count))))
    selected.frag.path <- file.path(out.frag.path, frag.files[tissue], 'fragments.replace_.tsv.gz')
    conv.ga <- convert.gene.activtity(selected.kidney.count, selected.frag.path, annotations, genome = 'mm10')
    conv.ga <- set.names(conv.ga, colname = paste(tissue, colnames(conv.ga), sep='-'))
    kidney.atac.ga.list[[tissue]] <- conv.ga
}

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, sett

In [37]:
# taking intersection of peaks among tissues
intersect.genes <- rownames(kidney.atac.ga.list[[1]])
for (i in 2:length(kidney.atac.ga.list)) {
    intersect.genes <- intersect(intersect.genes, rownames(kidney.atac.ga.list[[i]]))
}

for (tissue in names(kidney.atac.ga.list)) {
    kidney.atac.ga.list[[tissue]] <- kidney.atac.ga.list[[tissue]][intersect.genes ,]
}

kidney.atac.ga <- do.call(cbind, kidney.atac.ga.list)

In [38]:
print.names.match(colnames(kidney.atac.ga), names(kidney.atac.annot))

[1] "Length1, Length2, Length intersection, identical:"
[1] "27652 27652 27652 FALSE"


In [21]:
kidney.atac.ga <- kidney.atac.ga[, names(kidney.atac.annot)]

In [22]:
print.names.match(colnames(kidney.atac.ga), kidney.atac.annot)

[1] "Length1, Length2, Length intersection, identical:"
[1] "27652 27652 27652 TRUE"


In [40]:
print.names.match(rownames(kidney.rna.counts), rownames(kidney.atac.ga))

[1] "Length1, Length2, Length intersection, identical:"
[1] "31053 21807 20720 FALSE"


In [24]:
kidney.ga.path <- 'kidney/ATAC_gene_activities/'
write.counts.transpose(kidney.atac.ga, kidney.ga.path)
write.annotation(kidney.atac.annot, kidney.ga.path)

Warning message in dir.create(folder, recursive = TRUE):
"'kidney/ATAC_gene_activities' already exists"


## 5. Multi ATAC preprocess: fix peaks

In [42]:
scicar.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/sci-CAR/'
scicar.atac.counts <- readMM(file.path(scicar.path, 'GSM3271045_ATAC_mouse_kidney_peak_count.txt.gz'))
scicar.atac.barcode.table <- read.csv(file.path(scicar.path, 'GSM3271045_ATAC_mouse_kidney_cell.txt.gz'))
scicar.atac.genes.table <- read.csv(file.path(scicar.path, 'GSM3271045_ATAC_mouse_kidney_peak.txt.gz'))

In [43]:
scicar.atac.counts <- set.names(counts = scicar.atac.counts, colname = scicar.atac.barcode.table$sample, rowname = scicar.atac.genes.table$peak)

In [44]:
scicar.atac.counts

   [[ suppressing 31 column names 'coRNA-RNA-plate1-052.CGTAAGGAGT', 'coRNA-RNA-plate1-052.ACTCCGCCAA', 'coRNA-RNA-plate1-052.GGCTCGAGAT' ... ]]

   [[ suppressing 31 column names 'coRNA-RNA-plate1-052.CGTAAGGAGT', 'coRNA-RNA-plate1-052.ACTCCGCCAA', 'coRNA-RNA-plate1-052.GGCTCGAGAT' ... ]]

   [[ suppressing 31 column names 'coRNA-RNA-plate1-052.CGTAAGGAGT', 'coRNA-RNA-plate1-052.ACTCCGCCAA', 'coRNA-RNA-plate1-052.GGCTCGAGAT' ... ]]



252741 x 13395 sparse Matrix of class "dgTMatrix"
                                                                                        
GL456210.1-58882-59082     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456210.1-110303-110503   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456210.1-123592-123792   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456210.1-136519-136719   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456210.1-147592-147792   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456211.1-32396-32596     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456211.1-54840-55040     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456211.1-66755-66955     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456211.1-113668-113868   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
GL456211.1-162404-162604   . . . . . . . . . . . . . . . . .

## 6. Multi RNA preprocess

In [45]:
scicar.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/sci-CAR/'
scicar.rna.counts <- readMM(file.path(scicar.path, 'GSM3271044_RNA_mouse_kidney_gene_count.txt.gz'))
scicar.rna.barcode.table <- read.csv(file.path(scicar.path, 'GSM3271044_RNA_mouse_kidney_cell.txt.gz'))
scicar.rna.genes.table <- read.csv(file.path(scicar.path, 'GSM3271044_RNA_mouse_kidney_gene.txt.gz'))
scicar.rna.counts <- set.names(counts = scicar.rna.counts, colname = scicar.rna.barcode.table$sample, rowname = scicar.rna.genes.table$gene_id)

#### Check if feature sets have large overlap with single RNA

In [47]:
print.names.match(rownames(scicar.rna.counts), rownames(kidney.rna.counts))
# need to translate genes.

[1] "Length1, Length2, Length intersection, identical:"
[1] "49584 31053 0 FALSE"


In [48]:
sci.genes.nover <- gsub('\\.[0-9]+$', '', rownames(scicar.rna.counts))

In [49]:
sci.genes.conv <- mapIds(EnsDb.Mmusculus.v79,
       keys = sci.genes.nover,
       keytype = 'GENEID',
       column = 'SYMBOL')

Warning message:
"Unable to map 6893 of 49584 requested IDs."


In [50]:
sum(duplicated(sci.genes.conv))

[1] 6925

In [51]:
sum(is.na(sci.genes.conv))

[1] 6893

In [52]:
scicar.rna.counts <- set.names(counts = scicar.rna.counts, rowname = sci.genes.conv, allow.duplicate = TRUE)

In [53]:
inters.genes <- intersect(rownames(scicar.rna.counts), rownames(kidney.rna.counts))

In [54]:
# take intersection genes lest NA and duplicate.
scicar.rna.counts <- scicar.rna.counts[inters.genes, ]

In [55]:
print.names.match(rownames(scicar.rna.counts), rownames(kidney.rna.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "25767 31053 25767 FALSE"


## 7. Align multi ATAC RNA

In [56]:
print.names.match(colnames(scicar.rna.counts), colnames(scicar.atac.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "13893 13395 11296 FALSE"


In [57]:
scicar.cells <- intersect(colnames(scicar.rna.counts), colnames(scicar.atac.counts))
scicar.rna.counts <- scicar.rna.counts[, scicar.cells]
scicar.atac.counts <- scicar.atac.counts[, scicar.cells]

In [58]:
print.names.match(colnames(scicar.rna.counts), colnames(scicar.atac.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "11296 11296 11296 TRUE"


## 8. Map unimodal ATAC peaks to multi ATAC peaks

In [ ]:
kidney.atac.mapped.list <- list()
for (tissue in names(frag.files)) {
    selected.id <- which(kidney.atac.meta$samples == tissue)
    selected.kidney.count <- kidney.atac.counts[, selected.id]
    stopifnot(!any(duplicated(colnames(selected.kidney.count))))
    selected.frag.path <- file.path(out.frag.path, frag.files[tissue], 'fragments.replace_.tsv.gz')
    mapped.cnt <- map.peaks(selected.kidney.count, scicar.atac.counts, selected.frag.path, annotations)
    mapped.cnt <- set.names(counts = mapped.cnt, colname = paste(tissue, colnames(mapped.cnt), sep='-'))
    kidney.atac.mapped.list[[tissue]] <- mapped.cnt
}

Computing hash

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Extracting reads overlapping genomic regions



In [ ]:
# taking intersection of peaks among tissues
intersect.genes <- rownames(kidney.atac.mapped.list[[1]])
for (i in 2:length(kidney.atac.mapped.list)) {
    intersect.genes <- intersect(intersect.genes, rownames(kidney.atac.mapped.list[[i]]))
}

for (tissue in names(kidney.atac.mapped.list)) {
    kidney.atac.mapped.list[[tissue]] <- kidney.atac.mapped.list[[tissue]][intersect.genes ,]
}

kidney.atac.mapped <- do.call(cbind, kidney.atac.mapped.list)

In [69]:
print.count.dim(kidney.atac.mapped)

[1] "No NA."
[1] "No NA rows."
[1] "No NA cols."
[1] "Dimension of count:"
[1] 251727  27652


In [70]:
print.names.match(rownames(kidney.atac.mapped), rownames(scicar.atac.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "251727 252741 251727 FALSE"


## 9. Preprocess unimodal atac matched

In [84]:
mapped.anno.cells <- names(kidney.atac.annot)
kidney.atac.mapped <- kidney.atac.mapped[, mapped.anno.cells]

In [86]:
print.names.match(colnames(kidney.atac.mapped), names(kidney.atac.annot))

[1] "Length1, Length2, Length intersection, identical:"
[1] "27652 27652 27652 TRUE"


In [89]:
print.names.match(colnames(kidney.atac.mapped), colnames(kidney.atac.counts.uniq))

[1] "Length1, Length2, Length intersection, identical:"
[1] "27652 27652 27652 TRUE"


## 10. Align unimodal and multi ATAC

In [90]:
print.names.match(rownames(kidney.atac.mapped), rownames(scicar.atac.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "251727 252741 251727 FALSE"


In [91]:
scicar.atac.counts <- scicar.atac.counts[rownames(kidney.atac.mapped) ,]

In [ ]:
# single ATAC rematched
write.counts.transpose(kidney.atac.mapped, 'kidney/atac-mapped-peaks')
write.annotation(kidney.atac.annot, 'kidney/atac-mapped-peaks')
# sci-CAR RNA
write.counts.transpose(scicar.rna.counts, 'kidney/scicar/RNA/')
# sci-CAR ATAC
write.counts.transpose(scicar.atac.counts, 'kidney/scicar/ATAC/')